In [1]:
from MarkovNetwork import MarkovNetwork
import numpy as np
import pandas as pd
from tqdm import tqdm

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Binarizer

ACC_VISION_RANGE = 1
ACC_STEP_SIZE = 3
NUM_DIGITS_PER_GEN = 10

training_data = pd.read_csv('mnist.train.gz', sep=' ', compression='gzip', header=None)
training_data.rename(columns={0: 'class_label'}, inplace=True)
training_features = Binarizer(threshold=127).transform(training_data.drop('class_label', axis=1).values)
training_labels = training_data.loc[:, 'class_label'].values

creator.create('FitnessMulti', base.Fitness, weights=(-1.0, 1.0))
creator.create('Individual', list, fitness=creator.FitnessMulti)

def create_random_genome():
    """
        Use the Markov Network's default random genome generation process
        to generate new individuals
    """
    temp_mn = MarkovNetwork(num_input_states=13,
                            num_memory_states=10,
                            num_output_states=25,
                            probabilistic=False)
    return temp_mn.genome

toolbox = base.Toolbox()
toolbox.register('genome', create_random_genome)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.genome)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

def evaluate_acc(individual):
    """
        Evaluate the ACC individual on a subset of the training instances
    """
    X_train, _, y_train, _ = train_test_split(training_features, training_labels,
                                              stratify=training_labels,
                                              train_size=NUM_DIGITS_PER_GEN,
                                              random_state=all_gen_seeds[pbar.n])
    
    acc_fitness = 0.
    total_acc_steps = 0
    
    for (digit, digit_label) in zip(X_train, y_train):
        digit_grid = digit.reshape((28, 28))
        
        # Convert individual into MN
        acc = MarkovNetwork(num_input_states=13,
                            num_memory_states=10,
                            num_output_states=25,
                            probabilistic=False,
                            genome=individual)
        
        acc_X = np.random.randint(0, 28)
        acc_Y = np.random.randint(0, 28)
        acc_classifications = np.zeros(10, dtype=np.bool)
        
        for time_step in range(50):
            total_acc_steps += 1
            
            # Visual grid of the ACC
            visual_inputs = []
            for offset_X in range(-ACC_VISION_RANGE, ACC_VISION_RANGE + 1):
                for offset_Y in range(-ACC_VISION_RANGE, ACC_VISION_RANGE + 1):
                    try:
                        visual_inputs.append(digit_grid[acc_X + offset_X, acc_Y + offset_Y])
                    except IndexError:
                        visual_inputs.append(0)
            
            # Left-facing raycast sensor
            try:
                if acc_X <= 0:
                    raise IndexError
                visual_inputs.append(digit_grid[0:acc_X, acc_Y].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            # Right-facing raycast sensor
            try:
                if acc_X >= 27:
                    raise IndexError
                visual_inputs.append(digit_grid[(acc_X + 1):28, acc_Y].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            # Upper-facing raycast sensor
            try:
                if acc_Y <= 0:
                    raise IndexError
                visual_inputs.append(digit_grid[acc_X, 0:acc_Y].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            # Lower-facing raycast sensor
            try:
                if acc_Y >= 27:
                    raise IndexError
                visual_inputs.append(digit_grid[acc_X, (acc_Y + 1):28].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            acc.update_input_states(visual_inputs)
            acc.activate_network()
            
            # Outputs:
            #        0-9: classifications
            #        10-19: veto bits
            #        20: move up
            #        21: move down
            #        22: move left
            #        23: move right
            #        24: done bit
            acc_decisions = acc.get_output_states()
            acc_classifications = np.bitwise_and(acc_decisions[0:10],
                                                 np.bitwise_not(acc_decisions[10:20]))
            
            if acc_decisions[24]:
                break
            if acc_decisions[20]:
                acc_Y += ACC_STEP_SIZE
            if acc_decisions[21]:
                acc_Y -= ACC_STEP_SIZE
            if acc_decisions[22]:
                acc_X += ACC_STEP_SIZE
            if acc_decisions[23]:
                acc_X -= ACC_STEP_SIZE
        
        acc_digit_correct = acc_classifications[digit_label]
        if acc_digit_correct:
            acc_fitness += acc_digit_correct / np.sum(acc_classifications)
    
    return int(total_acc_steps / float(NUM_DIGITS_PER_GEN)), acc_fitness / float(NUM_DIGITS_PER_GEN)

def pareto_selection_operator(individuals, k):
    return tools.selNSGA2(individuals, int(k / 5.)) * 5

def mutation_operator(individual):
    # Point mutations
    num_mutations = int(0.0005 * len(individual))
    for byte_index in np.random.randint(0, len(individual), num_mutations):
        individual[byte_index] = np.random.randint(0, 256)
    
    # Duplication
    if len(individual) < 50000 and np.random.random() < 0.05:
        width = np.random.randint(15, 601)
        start_index = np.random.randint(0, len(individual) - width)
        insert_index = np.random.randint(0, len(individual))
        for val in reversed(individual[start_index:start_index + width]):
            individual.insert(insert_index, val)
    
    # Deletion
    if len(individual) > 1000 and np.random.random() < 0.02:
        width = np.random.randint(15, 601)
        start_index = np.random.randint(0, len(individual) - width)
        del individual[start_index:start_index + width]
    
    return individual,


toolbox.register('evaluate', evaluate_acc)
toolbox.register('mutate', mutation_operator)
toolbox.register('select', pareto_selection_operator)

def pareto_eq(ind1, ind2):
    return np.all(ind1.fitness.values == ind2.fitness.values)

total_gens = 10
pbar = tqdm(total=total_gens + 1)
all_gen_seeds = np.random.randint(low=1, high=1e9, size=total_gens + 1)

pop = toolbox.population(n=100)
hof = tools.ParetoFront(similar=pareto_eq)
stats = tools.Statistics(lambda ind: (int(ind.fitness.values[0]), round(ind.fitness.values[1], 3)))
stats.register('Minimum', np.min, axis=0)
stats.register('Maximum', np.max, axis=0)
stats.register('Progress', lambda x: pbar.update())

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0., mutpb=1.0, ngen=total_gens, 
                               stats=stats, halloffame=hof, verbose=False)
pbar.close()

100%|██████████| 11/11 [03:17<00:00, 17.42s/it]


In [7]:
testing_data = pd.read_csv('mnist.test.gz', sep=' ', compression='gzip', header=None, nrows=1000)
testing_data.rename(columns={0: 'class_label'}, inplace=True)
testing_features = Binarizer(threshold=127).transform(testing_data.drop('class_label', axis=1).values)
testing_labels = testing_data.loc[:, 'class_label'].values

def evaluate_acc_test(individual):
    """
        Evaluate the ACC individual on a subset of the training instances
    """
    
    acc_fitness = 0.
    total_acc_steps = 0
    
    for (digit, digit_label) in zip(testing_features, testing_labels):
        digit_grid = digit.reshape((28, 28))
        
        # Convert individual into MN
        acc = MarkovNetwork(num_input_states=13,
                            num_memory_states=10,
                            num_output_states=25,
                            probabilistic=False,
                            genome=individual)
        
        acc_X = np.random.randint(0, 28)
        acc_Y = np.random.randint(0, 28)
        acc_classifications = np.zeros(10, dtype=np.bool)
        
        for time_step in range(50):
            total_acc_steps += 1
            
            # Visual grid of the ACC
            visual_inputs = []
            for offset_X in range(-ACC_VISION_RANGE, ACC_VISION_RANGE + 1):
                for offset_Y in range(-ACC_VISION_RANGE, ACC_VISION_RANGE + 1):
                    try:
                        visual_inputs.append(digit_grid[acc_X + offset_X, acc_Y + offset_Y])
                    except IndexError:
                        visual_inputs.append(0)
            
            # Left-facing raycast sensor
            try:
                if acc_X <= 0:
                    raise IndexError
                visual_inputs.append(digit_grid[0:acc_X, acc_Y].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            # Right-facing raycast sensor
            try:
                if acc_X >= 27:
                    raise IndexError
                visual_inputs.append(digit_grid[(acc_X + 1):28, acc_Y].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            # Upper-facing raycast sensor
            try:
                if acc_Y <= 0:
                    raise IndexError
                visual_inputs.append(digit_grid[acc_X, 0:acc_Y].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            # Lower-facing raycast sensor
            try:
                if acc_Y >= 27:
                    raise IndexError
                visual_inputs.append(digit_grid[acc_X, (acc_Y + 1):28].max())
            except (ValueError, IndexError):
                visual_inputs.append(0)
            
            acc.update_input_states(visual_inputs)
            acc.activate_network()
            
            # Outputs:
            #        0-9: classifications
            #        10-19: veto bits
            #        20: move up
            #        21: move down
            #        22: move left
            #        23: move right
            #        24: done bit
            acc_decisions = acc.get_output_states()
            acc_classifications = np.bitwise_and(acc_decisions[0:10],
                                                 np.bitwise_not(acc_decisions[10:20]))
            
            if acc_decisions[24]:
                break
            if acc_decisions[20]:
                acc_Y += ACC_STEP_SIZE
            if acc_decisions[21]:
                acc_Y -= ACC_STEP_SIZE
            if acc_decisions[22]:
                acc_X += ACC_STEP_SIZE
            if acc_decisions[23]:
                acc_X -= ACC_STEP_SIZE
        
        acc_digit_correct = acc_classifications[digit_label]
        if acc_digit_correct:
            acc_fitness += acc_digit_correct / np.sum(acc_classifications)
    
    return int(total_acc_steps / float(len(testing_labels))), acc_fitness / float(len(testing_labels))

for ind in hof:
    print(evaluate_acc_test(ind))

(1, 0.10649999999999982)
